In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ta

sns.set_theme()

In [13]:
data = pd.read_csv('aapl_5m_train.csv').dropna()
data.head()


,Unnamed: 0,Timestamp,Gmtoffset,Datetime,Open,High,Low,Close,Volume
0,0,1609770600,0,2021-01-04 14:30:00,133.570007,133.611602,132.389999,132.809997,6624663.0
1,1,1609770900,0,2021-01-04 14:35:00,132.750000,132.750000,131.809997,131.889999,2541553.0
2,2,1609771200,0,2021-01-04 14:40:00,131.500000,132.339996,131.500000,132.059997,2492415.0
3,3,1609771500,0,2021-01-04 14:45:00,132.000000,132.250000,131.899993,132.250000,1859131.0
4,4,1609771800,0,2021-01-04 14:50:00,132.000000,132.018096,131.520004,131.589996,1780105.0


In [14]:
rsi_indicator = ta.momentum.RSIIndicator(data.Close, window = 100)
data['rsi'] = rsi_indicator.rsi()

In [15]:
historic = data.dropna()
historic['buy_signal'] = historic.rsi < 20
historic['sell_signal'] = historic.rsi > 80

C:\Users\Alejandro\AppData\Local\Temp\ipykernel_12432\3384102837.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  historic['buy_signal'] = historic.rsi < 20
C:\Users\Alejandro\AppData\Local\Temp\ipykernel_12432\3384102837.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  historic['sell_signal'] = historic.rsi > 80


In [16]:
historic.head()

,Unnamed: 0,Timestamp,Gmtoffset,Datetime,Open,High,Low,Close,Volume,rsi,buy_signal,sell_signal
99,99,1609863300,0,2021-01-05 16:15:00,130.335006,130.335006,129.929992,130.130004,1636275.0,47.265461,False,False
100,100,1609863600,0,2021-01-05 16:20:00,130.125000,130.199798,130.010101,130.100006,868647.0,47.168818,False,False
101,101,1609863900,0,2021-01-05 16:25:00,130.090103,130.460006,130.029998,130.380004,1216100.0,48.168013,False,False
102,102,1609864200,0,2021-01-05 16:30:00,130.374893,130.429992,130.309997,130.380004,875771.0,48.168013,False,False
103,103,1609864500,0,2021-01-05 16:35:00,130.382904,130.679992,130.320007,130.464996,1063531.0,48.469851,False,False


In [17]:
COM = 0.125/100
SL = 0.05
TP = 0.05
BORROW_RATE = 0.25/100

cash = 1_000_000
n_shares = 50

In [18]:
from dataclasses import dataclass

@dataclass

class Operation:
    time: str
    price : float
    stop_loss: float
    take_profit: float
    n_shares: int
    type: str


In [19]:
active_positions = list[Operation]=[]

for i, row in historic.iterrows():
    # ---BUY---
    # Check signal 
    if not row.buy_signal:
        continue
    # Do we have enough cash
    if cash < row.Close * n_shares * (1 + COM):
        continue
# Discount the cost
    cash-= row.Close * n_shares * (1 + COM)
# Save the operation as active position
    active_positions.append(
        Operation(
        time = row.Datetime,
        price = row.Close,
        take_profit= row.Close * (1 + TP),
        stop_loss= row.Close * (1 - SL),
        n_shares = n_shares,
        type="LONG"
        )
    )

TypeError: 'type' object does not support item assignment